# NLE Assessed Coursework 1

For this assessment, you are expected to complete and submit this notebook file.  When answers require code, you may import and use library functions (unless explicitly told otherwise).  All of your own code should be included in the notebook rather than imported from elsewhere.  Written answers should also be included in the notebook.  You should insert as many extra cells as you want and change the type between code and markdown as appropriate.

In order to avoid misconduct, you should not talk about these coursework questions with your peers.  If you are not sure what a question is asking you to do or have any other questions, please ask me or one of the Teaching Assistants.

Marking guidelines are provided as a separate document.

In order to provide unique datasets for analysis by different students, you must enter your candidate number in the following cell.

In [6]:
candidateno=184514 #this MUST be updated to your candidate number so that you get a unique data sample


In [309]:
#preliminary imports
import sys
sys.path.append(r'\\ad.susx.ac.uk\ITS\TeachingResources\Departments\Informatics\LanguageEngineering\resources')
sys.path.append(r'/Users/juliewe/resources')
sys.path.append(r'/Users/Bayley/Documents/resources')

import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import zip_longest
from nltk.classify.api import ClassifierI
from nltk.tokenize import word_tokenize

from sussex_nltk.corpus_readers import AmazonReviewCorpusReader
import random
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Bayley\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Bayley\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Bayley\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Question 1: Naive Bayes Classification (25 marks)

In this question, you will be considering how a Naive Bayes classifier can be applied to the task of deciding whether sentences are relevant or not relevant to the kitchen domain.

The code below will generate for you two small unique sets of sentences, which you should refer to in your answer to this question.   This question will be marked on the quality of your explanations rather than the quality of your code.

In [310]:
#Do NOT change the code in this cell.

topics=["book","kitchen","dvd","electronics"]
samplesize=20
stop=stopwords.words('english')

trainingsentences=[]
testsentences=[]
cr = AmazonReviewCorpusReader()

for topic in topics:
    random.seed(candidateno)
    if topic == "kitchen":
        key="kitchen"
    else:
        key="not-kitchen"
    topicsentences=[]
    while len(topicsentences)<10:
        topicsentences=[({token.lower():True for token in doc if token not in stop and token.isalpha()},key) for doc in cr.category(topic).sample_sents(samplesize=samplesize) if len(doc)>0]
    testsentences+=topicsentences[0:3]
    trainingsentences+=topicsentences[3:]

random.shuffle(trainingsentences)
mixup=testsentences[:4]+testsentences[5:]
random.shuffle(mixup)
testsentences=[testsentences[4]]+mixup
print("Number of training sentences is {}.  Number of testing sentences is {}".format(len(trainingsentences),len(testsentences)))


Number of training sentences is 49.  Number of testing sentences is 12


In [311]:
def class_prior_prob(training_data):
    kitchen_count = 0
    not_kitchen_count = 0
    dict = {}
    for d in training_data:
        if(d[1] == 'kitchen'):
            kitchen_count += 1
        if(d[1] == 'not-kitchen'):
            not_kitchen_count +=  1
    dict['kitchen'] = kitchen_count / len(training_data)
    dict['not-kitchen'] = not_kitchen_count / len(training_data)
    total = kitchen_count + not_kitchen_count
    print("Number of kitchen documents:",str(kitchen_count))
    print("Number of non-kitchen documents:",str(not_kitchen_count))

    return dict

In [312]:
def vocab_known(training_data):
    words_known = set()
    for phrase in training_data:
        for word in phrase[0]:
            words_known.add(word)
    
    return words_known

In [313]:
def conditional_prob(training_data):
    kitchen_count = {}
    not_kitchen_count = {}
    result = {}
    i = 0 #counter
    vocabulary_known = list(vocab_known(training_data))
    for data in training_data:
        if data[1] == 'kitchen':
            for phrase in data[0]:
                for word in phrase.split():
                    kitchen_count[word] = (kitchen_count.get(word,0)+1)
                    i += 1
    for word in vocabulary_known:
        if word in kitchen_count.keys():
            i += 1
            kitchen_count[word] += 1
        else:
            kitchen_count[word] = 1
            i += 1
    for key,value in kitchen_count.items():
        kitchen_count[key] = (value / i)
    print(kitchen_count)
    print(i)
    
    i = 0
    for data in training_data:
        if data[1] == 'not-kitchen':
            for phrase in data[0]:
                for word in phrase.split():
                    not_kitchen_count[word] = (not_kitchen_count.get(word,0)+1)
                    i += 1
    for word in vocabulary_known:
        if word in not_kitchen_count.keys():
            i += 1
            not_kitchen_count[word] += 1
        else:
            not_kitchen_count[word] = 1
            i += 1
    for key,value in not_kitchen_count.items():
        not_kitchen_count[key] = (value / i)
    
    result['not-kitchen \n'] = not_kitchen_count
    result['kitchen \n'] = kitchen_count
    return result

In [314]:
def classify(d, priors, c_probs):
    label=""
    probs_kitchen = 0
    probs_not_kitchen = 0
    words_known = vocab_known(training_data)
    for word in d:
        if word[0] in words_known:
            for word in d:
                if word[0] in c_probs['kitchen'].keys():
                    probs_kitchen =+ math.log10(c_probs['kitchen'][word[0]])
            for word in d:
                if word[0] in c_probs['not-kitchen'].keys():
                    probs_kitchen =+ math.log10(c_probs['not-kitchen'][word[0]])
                    
            probs_kitchen =+ math.log10(priors['kitchen'])
            probs_not_kitchen =+ math.log10(priors['not-kitchen'])
            
            if probs_kitchen > probs_not_kitchen:
                label = "kitchen"
            elif probs_kitchen < probs_not_kitchen:
                label = "not-kitchen"
            else:
                label = random.choice(['kitchen'],['not-kitchen'])
    return label

In [315]:
import math
import random
class NBClassifier(ClassifierI):
    def __init__(self):
        self.labels=["P","N"]
        pass
    def labels(self):
        return self._labels
    
    def _set_known_vocabulary(self,training_data):
        known = []
        
        for (d, label) in training_data:
            known+=list(d.keys())
            
        self.known = set(known)
    def _set_priors(self,training_data):
        priors_probs={}
        
        for (d, label) in training_data:
            priors_probs[label] = priors_probs.get(label,0) + 1 #smoothing
        total=sum(priors_probs.values())
        
        for (key,value) in priors_probs.items():
            priors_probs[key]=value/total
            
        self.priors_probs=priors_probs
    def _set_cond_probs(self, training_data):
        cond_probs={}
        
        for (d, label) in training_data:
            class_cond = cond_probs.get(label,{})
            
            for word in d.keys():
                class_cond[word] = class_cond.get(word,0) + 1 #smoothing
            
            cond_probs[label] = class_cond
    
        for (label, class_cond) in cond_probs.items():
            
            for word in self.known:
                class_cond[word] = class_cond.get(word,0) + 1
            cond_probs[label] = class_cond
            
        for (label, distance) in cond_probs.items():
            total = sum(distance.values())
            cond_probs[label] = {key:value/total for (key, value) in distance.items()}
        
        self.cond_probs = cond_probs
    def train(self, training_data):
        self._set_known_vocabulary(training_data)
        self._set_priors(training_data)
        self._set_cond_probs(training_data)
    
    def classify(self, d):
        d_probs = {key:math.log(value) for (key,value) in self.priors_probs.items()}
        for word in d[0]:
            if word in self.known:
                d_probs = {classlabel:sofar+math.log(self.cond_probs[classlabel].get(word,0)) for (classlabel,sofar) in d_probs.items()}
                
        highprobs = max(d_probs.values())
        classes = [c for c in d_probs.keys() if d_probs[c] == highprobs]
        return random.choice(classes)

a) With reference to the sentences generated above, **explain** how a Naive Bayes classifer would be trained to carry out the task of deciding whether sentences are relevant to the kitchen domain.  You do **not** need to build or train a classifier.  However, you should explain the relevant probabilities with reference to examples taken from your samples of sentences.  \[10 marks\]

### Assumptions Made

1. Each document is treated as a **bag-of-words**, meaning that the word order and grammatical structure is ignored. Preprocessing here is critical, as it helps constitute what a word is. Through **feature extraction**, words are treated as features, so words within documents provide evidence of being in a certain class. 

2. Use of **multi-variate Bernouilli event model**.This is the frequency of a word in a document, isn't considered, just whether a word occurs or not (*binary-valued* representation checking if word occurs). Training documents will be presented as a pair, a dictionary that maps each word that appears to True, and a string denoting the class of the document.

### How the classifier works

To train data, is to manually specifying which class the sentence belongs to. The AmazonCorpus specifies what class the sentence belongs to, and is used to create the training/testing lists. Training sentences will be given either the class 'Kitchen' or 'Not Kitchen', and these will be compared to the testing sentences to decide what class the testing sentences belong to. In order to classify a document, $d$, we need to determine which of the probabilities is greater:

$$P(\,\mbox{kitchen}\,|\,d) \qquad\qquad \mbox{versus} \qquad\qquad P(\,\mbox{not kitchen}\,|\,d)$$

where $d$ is the document.

$P(X|Y)$ means the probability of $X$ given $Y$. So, $P(\,\mbox{kitchen}\,|\,d)$ means the probability, given a document $d$, of $d$ being of class `kitchen`.
To calculate the probabilities, Bayes' rule can be applied, leading to the following comparison:

$$P(\,d\,|\,\mbox{kitchen}\,)\cdot P(\,\mbox{kitchen}\,) \qquad\qquad \mbox{versus} \qquad\qquad P(\,d\,|\,\mbox{not kitchen}\,)\cdot P(\,\mbox{not kitchen}\,)$$

Here $P(\,d\,|\,\mbox{kitchen}\,)$  is the probability of a document in the `kitchen` category being the document $d$ and, $P(\,\mbox{kitchen}\,)$ is the probability of a randomly selected document being of category `kitchen`.

### Class Priors

As stated , $P(\,\mbox{kitchen}\,)$ is the probability of a randomly selected document being of category `kitchen`, where as $P(\,\mbox{not kitchen}\,)$ is the probability of a randomly selected document being of category `not kitchen`. This is called the class priors. We can obtain estimated values for these probabilities using the training data. If the training data consisted of $n_1$ documents of class `kitchen` and $n_2$ documents of class `not kitchen`, then we can represent the probabilities as:

$$P(\mbox{Kitchen})=\frac{n_1}{n_1+n_2} \qquad \mbox{and} \qquad P(\mbox{not Kitchen}) = \frac{n_2}{n_1+n_2}$$
Where $n_1$ is the number of documents in the training documents classed as `Kitchen` and $n_2$ documents of class `not Kitchen`.
The code below computes the number of documents in the training data that relate to each class, `kitchen` and `not-kitchen`


In [316]:
print("Training sentences:")
train_prior_probs = class_prior_prob(trainingsentences)
print("Testing sentences:")
test_prior_probs = class_prior_prob(testsentences)

Training sentences:
Number of kitchen documents: 12
Number of non-kitchen documents: 37
Testing sentences:
Number of kitchen documents: 3
Number of non-kitchen documents: 9


From the training data above, `P(kitchen)` will have a prior probability of 12 / 12 + 37, which equals:

$$P(\mbox{kitchen})=\frac{12}{49} \qquad \mbox{≃} \qquad \mbox{0.24 (2.d.p)}$$

And `P(not kitchen)` will have a prior probability of 37 / 12 + 37 which equals:

$$P(\mbox{not kitchen})=\frac{37}{49} \qquad \mbox{≃} \qquad \mbox{0.76 (2.d.p)}$$

The code below computes estimates of the class priors from the training data, which is expressed in the table below which prints the prior probability of the documents in the training data for each class.

In [317]:
df = pd.DataFrame(train_prior_probs.items(),columns=['Class','Prior Probability'])
display(df)

,Class,Prior Probability
0,kitchen,0.244898
1,not-kitchen,0.755102


The below the code, shows the class priors for the test sentnces given. As we know from above, there are 3 kitchen documents and 9 not-kicthen documents. This gives a total of 12 documents:

$$P(\mbox{kitchen})=\frac{9}{12} \qquad \mbox{=} \qquad \mbox{0.25 (2.d.p)} \qquad \mbox{and} \qquad P(\mbox{not-kitchen})=\frac{9}{12} \qquad \mbox{=} \qquad \mbox{0.75 (2.d.p)}$$

In [318]:
df = pd.DataFrame(test_prior_probs.items(),columns=['Class','Prior Probability'])
display(df)

,Class,Prior Probability
0,kitchen,0.25
1,not-kitchen,0.75


### Conditional Probabilities

For each feature, we wish to know the probability of occurrence for each class:

$$P(\,d\,|\,\mbox{kitchen}\,)$$  $$P(\,d\,|\,\mbox{not kitchen}\,)$$

In order to **estimate** the probability, we know the label of each document **label(di)** and features of each document **feats(di)**. The problem we have is that $d$ is a document, that we won't have seen in the training data. 
The code below computes estimates of the conditional probability of a word given a class from training data.


In [319]:
train_cond_probs = conditional_prob(trainingsentences)
test_cond_probs = conditional_prob(testsentences)

{'i': 0.01642710472279261, 'believe': 0.004106776180698152, 'bacteria': 0.004106776180698152, 'growth': 0.004106776180698152, 'answer': 0.004106776180698152, 'making': 0.004106776180698152, 'espresso': 0.004106776180698152, 'easy': 0.004106776180698152, 'enough': 0.004106776180698152, 'follow': 0.004106776180698152, 'instructions': 0.004106776180698152, 'watch': 0.004106776180698152, 'video': 0.004106776180698152, 'think': 0.004106776180698152, 'well': 0.004106776180698152, 'insulated': 0.004106776180698152, 'turn': 0.004106776180698152, 'water': 0.004106776180698152, 'cools': 0.004106776180698152, 'faster': 0.004106776180698152, 'would': 0.004106776180698152, 'ca': 0.004106776180698152, 'say': 0.004106776180698152, 'really': 0.004106776180698152, 'drawback': 0.004106776180698152, 'adding': 0.004106776180698152, 'porcelain': 0.004106776180698152, 'enamel': 0.004106776180698152, 'looks': 0.004106776180698152, 'good': 0.004106776180698152, 'multiple': 0.004106776180698152, 'benefits': 0.

The Naïve Bayes model of document classification addresses this by making a major simplifying assumption, that the probability that different words occur in a document are independent of one another. An example shown below:

\begin{eqnarray*}
P(\,\mbox{"Isolation from outside"}\,|\,\mbox{kitchen}\,) &=& P(\{\mbox{"Isolation"},\mbox{"from"},\mbox{"outside"},\}\,|\,\mbox{kitchen}\,)\\
&=& P(\,\mbox{kitchen}\,)\times P(\mbox{"Isolation"}\,|\,\mbox{kitchen}\,)\times P(\mbox{"from"}\,|\,\mbox{kitchen}\,)\times P(\mbox{"outside"}\,|\,\mbox{kitchen}\,)
\end{eqnarray*}

To estimate conditional probabilities, we take the number of times words occur in `kitchen` along with all words that are defined as the class `kitchen`. The probability that a word occurs in the classis the number of times it occurs divided by the number of words defined as part of the class **kitchen**



b) With reference to the sentences generated above, **explain** how a trained Naive Bayes classifier would assign a class to a sentence. \[5 marks\]

In order to classify a sentence, the probability of the sentence being in the `kitchen` class given the sentence is compared to the probability it is in the `not-kitchen` class. The higher probability , determines the  class given. 

If a sentence has equal probability of being either class, for example:

$$P(\,\mbox{kitchen}) \qquad\qquad \mbox{=} \qquad\qquad P(\,\mbox{not kitchen})$$

Then the classifier would randomly choose one of the classes using a random function, as the sentence is no more one class than another.


### Add one smoothing

Some words many occur in the `kitchen` class, but may not occur at all in the `not-kitchen` class. This leads to an issue when calculating the conditional probabilities for documents. By not occurring in the `not kitchen` , the probability  would be 0. As we are multiplying probabilities together, the probability that the entire document occurs in the `not kitchen` class would be 0, which is wrong.

To resolve this issue, we use **add one smoothing**. This is adds one to the count of all words in the known vocabulary, meaning the minimum would be 1, not 0. This fixes the previous problem stated as it would not reduce the probability that the entire document occurs in the `not kitchen` class would be 0, leaving a valid probability.
The **known vocabulary** is the collection of all the words that occur in the training data.

c) The labels assigned by a Naive Bayes classifier to the sentences contained in the variable `testsentences` above are \['kitchen','not-kitchen','not-kitchen','not-kitchen','kitchen','not-kitchen','kitchen','not-kitchen','not-kitchen','not-kitchen','kitchen','not-kitchen'\].  Using this example to illustrate your answer, **explain** how each of the performance metrics of *accuracy*, *precision*, *recall* and *F1-score* are calculated.  Which of these metrics would you use to choose between classification models in this example?  **Justify your answer**. \[10 marks\]


Below, I have printed the classes given to the test sentences. In the question it states that there are meant to be 4 test sentences with the class `kitchen` and 8 with the class `not-kitchen`. The data below conveys that the classifier has identified **3** `kitchen` sentences and **9** `not-kitchen` sentences. This shows that the classifier is not 100% *accurate*, as it has classified atleast 1 test sentence incorrectly.

In [320]:
x = NBClassifier()
x.train(trainingsentences)
for i in testsentences:
    print(i[1])

kitchen
not-kitchen
not-kitchen
not-kitchen
kitchen
kitchen
not-kitchen
not-kitchen
not-kitchen
not-kitchen
not-kitchen
not-kitchen


## Evaluating Classifiers
In order to evaluate classifiers, we can calculate the accuracy, precision, recall and F1-Score. These allow to see how good a classifier would be when classifying  **unseen documents**, this is a label dataset that has not been used in training. To help demonstrate the accuracy and error rate, a confusion matrix can be used. This maps: 
- True positives **(TP)**: Should be labelled 'kitchen' and was!
- True negatives **(TN)**: Should be labelled 'kitchen', but labelled 'not-kitchen'.
- False positives **(FP)**: Should have been labelled 'not-kitchen' but labelled 'kitchen'.
- False negatives **(FN)**: Should be labelled 'not-kitchen' and was!
In addition, the total number of sentences / data items is represented using **N**.

The code below is a function that calculates the **accuracy or error rate** of a classifier & prints the **Confusion matrix**, or produces a **classification report**, which consists of the precision, recall, accuracy and f1.score for both classes, `kitchen` and`not kitchen`, by supplying the parameters:

1. Accuracy Score
2. Error Rate 
3. Classification Report

In [321]:
def resultPrinter(x):
    classifier1 = NBClassifier()
    classifier1.train(trainingsentences)
    predictedClasses = []
    classifier1.classify_many(testsentences)

    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import classification_report
    from sklearn.metrics import accuracy_score
    actualClasses = []
    for i in testsentences:
        actualClasses.append(i[1])
        
    results = confusion_matrix(actualClasses, classifier1.classify_many(testsentences))
    print('Confusion Matrix:\n')
    print(results, '\n')
        
    if (x== 1 ):
        print('Accuracy:', accuracy_score(actualClasses, classifier1.classify_many(testsentences)))
    if (x== 2 ):
        print('Error Rate:', 1 - accuracy_score(actualClasses, classifier1.classify_many(testsentences)))
    if (x== 3):
        print('Classification Report:\n\n', classification_report(actualClasses, classifier1.classify_many(testsentences)))
        

### Accuracy
The accuracy is the proportion of items in a test that are classified correctly.
To calculate **accuracy** the forumla is followed:

$$\frac{\mbox{number of test documents that the classifier classifies correctly (TP + TN)}}
{\mbox{total number of test documents (N)}}$$

For example, if all twelve of the test sentences where classified correctly, then the *accuracy* would be 1. Where as if half of the sentences were classified correctly, then the classifier would have an accuracy of only 0.5. The accuracy of my classifier is calculated below:

In [322]:
resultPrinter(1)

Confusion Matrix:

[[1 2]
 [1 8]] 

Accuracy: 0.75


The confusion matrix is printed showing the values of TP, FP, FN and TN. We can work out the value of N by adding all of these values together. We can check if the accuracy has been calculated correctly by subsituting the values for TP and TN into the formula above. This accuracy means that the classifier is good at classifying documents correctly. A high accuracy score tends to be desireable.

$$\mbox{Accuracy}=\frac{1+8}{1+8+2+1} \qquad \mbox{=} \qquad  \frac{9}{12} \qquad  \mbox{or} \qquad  \mbox{0.75 (2.d.p)} $$

### Error rate
The error rate is the proportion of items in the test that are classified incorrectly.
To calculate the *Error rate*:

$$\frac{\mbox{number of test documents that the classifier classifies incorrectly (FP + FN)}}
{\mbox{total number of test documents (N)}}$$

This could also be calculated by doing **1 - accuracy**. For example, if all twelve of the test sentences where classified incorrectly, then the *error rate* would be 1. Where as if half of the sentences were classified incorrectly, then the classifier would have an error rate of only 0.5. The error rate of my classifier is calculated below.

In [323]:
resultPrinter(2)

Confusion Matrix:

[[1 2]
 [1 8]] 

Error Rate: 0.25


Calculation checked by subsituting the values into the formula above. The error rate calculated supports that the classifier is good at classifying documents correctly, since a low error rate is desireable.

$$\mbox{Accuracy}=\frac{2+1}{1+8+2+1} \qquad \mbox{=} \qquad  \frac{3}{12} \qquad  \mbox{or} \qquad  \mbox{0.25 (2.d.p)} $$

### Precision
This is the proportion of `kitchen` predictions that are correct, and is calculated with the forumla below:

\begin{eqnarray*}
P=\frac{TP}{TP + FP}
\end{eqnarray*}

For example, if 3 of the 4 `kitchen` items are predicted correctly and 2 of the `not-kitchen` are predicted incorrectly, the precision is 3/3+2, which is 0.6.

### Recall
This is the proportion of actual `kitchen` features/items that were predicted correctly :
\begin{eqnarray*}
R=\frac{TP}{TP + FN}
\end{eqnarray*}

For example, if three of the four `kitchen` items are predicted correctly the recall is 3/3+1, which is 0.75.

### F1-Score

We want high **recall & precision**. We combine: precision (P) & recall (R) scores using `F1-Score`.

\begin{eqnarray*}
F1 = \frac{2\times P\times R}{P+R}
\end{eqnarray*}

Using the example recall and precision scores from above, the F1-Score would be (2 * 0.6 * 0.75) / (0.6 + 0.75) = 2/3 or 0.66 (2.d.p).

The F1-score is the harmonic mean of the precision and recall. This means that it will always *sway* closer to the lower value of the two.

The code below prints a copy of the confusion matrix, along with values for the precision, recall and f1-score for each class.

In [324]:
resultPrinter(3)

Confusion Matrix:

[[1 2]
 [1 8]] 

Classification Report:

               precision    recall  f1-score   support

     kitchen       0.50      0.33      0.40         3
 not-kitchen       0.80      0.89      0.84         9

    accuracy                           0.75        12
   macro avg       0.65      0.61      0.62        12
weighted avg       0.72      0.75      0.73        12



Lets check that the precision, recall and f1-score values of class `kitchen` have been correctly computed.
\begin{eqnarray*}
P=\frac{TP}{TP + FP} = \frac{1}{1 + 1} = 0.5 (2.d.p)
\end{eqnarray*}

\begin{eqnarray*}
R=\frac{TP}{TP + FN} = \frac{1}{1 + 2} = 0.33 (2.d.p)
\end{eqnarray*}

\begin{eqnarray*}
F1 = \frac{2\times P\times R}{P+R} = \frac{2* 0.5* 0.33}{0.5 + 0.33} = 0.40 (2.d.p)
\end{eqnarray*}

We want a value for how accurate a classification model is at identifying whether a sentence is part of the `kitchen` class or not, we should use the **F1-Score**.

This is because it gives an harmonic-mean of precision and recall, returning a reliable value. Using the precision/recall score alone,  only takes into account the proportion of correct predictions or the proportion of items predicted correctly, where as we want to take both into account.

Accuracy isn't great at comparing the classifiers as it doesnt take into account if a classifier may be accurate at specifying one class correctly, rather than all classes. **E.g.** The results show the classifier coded could be good at predicting the `not-kitchen` class, but not the `kitchen` class

## Question 2: Training Data for Sentiment Analysis (25 marks)
The objective of this question is to investigate the extent to which performance of a Naive Bayes classifier is affected by the quantity and quality of the training data.  Does more training data mean better performance?  Is performance degraded if we train on one domain and test on another domain?  For example, suppose we train a sentiment classifier on book reviews and then test that classifier on a collection of dvd reviews. Does it perform as well as it would when trained on dvd reviews?

The code below is included to enable you to get pre-formatted training and test data for a given category (evenly split between positive and negative reviews).  In this question, there are marks available for the quality of your programming, your experimental design and your interpretation of results.


In [7]:
def split_data(data, ratio=0.7): # when the second argument is not given, it defaults to 0.7
    """
    Given corpus generator and ratio:
     - partitions the corpus into training data and test data, where the proportion in train is ratio,

    :param data: A corpus generator.
    :param ratio: The proportion of training documents (default 0.7)
    :return: a pair (tuple) of lists where the first element of the 
            pair is a list of the training data and the second is a list of the test data.
    """
    
    data = list(data)  
    n = len(data)  
    train_indices = random.sample(range(n), int(n * ratio))          
    test_indices = list(set(range(n)) - set(train_indices))    
    train = [data[i] for i in train_indices]           
    test = [data[i] for i in test_indices]             
    return (train, test)                       
 

def feature_extract(review):
    """
    Generate a feature representation for a review
    :param review: AmazonReview object
    :return: dictionary of Boolean features
    """
    return {word:True for word in review.words()}

def get_training_test_data(category,ratio=0.7,seed=candidateno):
    """
    Get training and test data for a given category and ratio, pre-formatted for use with NB classifier
    :param category: category of review corpus, one of ["kitchen, "dvd, "book", "electronics"]
    :param ratio: proportion of data to use as training data
    :return: pair of lists 
    """
    reader=AmazonReviewCorpusReader().category(category)
    random.seed(candidateno)
    pos_train, pos_test = split_data(reader.positive().documents(),ratio=ratio)
    neg_train, neg_test = split_data(reader.negative().documents(),ratio=ratio)
    train_data=[(feature_extract(review),'P')for review in pos_train]+[(feature_extract(review),'N') for review in neg_train]
    test_data=[(feature_extract(review),'P')for review in pos_test]+[(feature_extract(review),'N') for review in neg_test]
    return train_data,test_data


a) By varying the amount of training data used, **investigate** the impact of the amount of training data used on the accuracy of a Naive Bayes classifier for each of the four domains: *dvd*, *book*, *kitchen* and *electronics*.  You should use the NaiveBayesClassifier from the `nltk.classify` library.  You should also use a table and an appropriate graph(s) to display your results.  Make sure you **discuss** your results and conclusions. \[8 marks\]

### Experimental Design

In order to investigate how the amount of training data impacts the accuracy of a classifier, I have decided test the accuracy at four evenly spaced out ratios of 0.2, 0.4, 0.6 and 0.8. From these range of values, the tests should be able to indicate whether much less training data, i.e. ratio of 0.2, differs much in accuracy compared to a higher ratio of 0.8. For each feature, four accuracies will be calculated, one for each ratio, using the same testing data. These accuracys will be stored in an array, in order from ratio 0.2, to ratio 0.8, so it can be presented in a table and graph clearly.

In [5]:
'''
Creating arrays to hold accuracy for each ratio per domain 
I have chosen to use four different ratios to change the size of the training data for each domain.
This takes a while to load, so please be patient
'''
import nltk.classify.naivebayes
import numpy as np
NB = nltk.NaiveBayesClassifier
dvda = []
booka = []
kita = []
eleca = []
features = {"dvd", "book", "kitchen", "electronics"}
accs = {0.2, 0.4, 0.6, 0.8}
for feature in features:
        train_data1, testing_data1 = get_training_test_data(feature, 0.2)
        train_data2, testing_data1 = get_training_test_data(feature, 0.4)
        train_data3, testing_data1 = get_training_test_data(feature, 0.6)
        train_data4, testing_data1 = get_training_test_data(feature, 0.8)
        accuracy_1 = nltk.classify.accuracy(NB.train(train_data1), testing_data1)
        accuracy_2 = nltk.classify.accuracy(NB.train(train_data2), testing_data1)
        accuracy_3 = nltk.classify.accuracy(NB.train(train_data3), testing_data1)
        accuracy_4 = nltk.classify.accuracy(NB.train(train_data4), testing_data1)
        if (feature == "dvd"):
            dvda.append(accuracy_1)
            dvda.append(accuracy_2)
            dvda.append(accuracy_3)
            dvda.append(accuracy_4)
        elif (feature == "book"):
            booka.append(accuracy_1)
            booka.append(accuracy_2)
            booka.append(accuracy_3)
            booka.append(accuracy_4)
        elif (feature == "kitchen"):
            kita.append(accuracy_1)
            kita.append(accuracy_2)
            kita.append(accuracy_3)
            kita.append(accuracy_4)
        elif (feature == "electronics"):
            eleca.append(accuracy_1)
            eleca.append(accuracy_2)
            eleca.append(accuracy_3)
            eleca.append(accuracy_4)
            
            
data = [['DVD', dvda[0], dvda[1], dvda[2], dvda[3]],
        ['Book', booka[0], booka[1], booka[2], booka[3]],
        ['Kitchen',kita[0],kita[1],kita[2],kita[3]], 
        ['Electronics', eleca[0],eleca[1],eleca[2],eleca[3]]]
        
df = pd.DataFrame(data,columns=['Domain','Accuracy at ratio 0.2', 'Accuracy at ratio 0.4', 'Accuracy at ratio 0.6', 'Accuracy at ratio 0.8'])
display(df)

barWidth = 0.2
fig, ax = plt.subplots()

b = np.arange(len(dvda))
x1 = ax.bar(b, dvda, barWidth, bottom=0)
x2 = ax.bar(b + barWidth, booka, barWidth, bottom=0)
x3 = ax.bar(b + barWidth*2, kita, barWidth, bottom=0)
x4 = ax.bar(b + barWidth*3, eleca, barWidth, bottom=0)

ax.set_title('Impact of amount of training data on accuracy per domain')
ax.set_xticks(b + barWidth)
ax.set_xticklabels(('0.2', '0.4', '0.6', '0.8'))

ax.legend((x1[0], x2[0], x3[0], x4[0]), ('DVD', 'Book', 'Kitchen', 'Electronics'))
ax.set_ylabel('Accuracy')
ax.set_xlabel('Ratio')
ax.autoscale_view()
plt.rcParams["figure.figsize"] = [20,9]
plt.show()


NameError: name 'get_training_test_data' is not defined

### Results discussion

**Distrubution of accuracy (DOA)** = **highest accuracy - lowest accuracy**

We wish for a small Distribution of accuracy, along with overall high accuracies. This is because we want a classifier that can classify items accurately to all classes/domains.

From the table and graph, the results show that by varying the amount of training data, the accuracy of the classifier per class does affect the distrubution of accuracys.

For 0.2, there is not much distribution between classes, other than `book` being slightly less accurate than the rest. This means for a small amount of data, the classifier is less accurate for classifying the `book` class. The *DOA* for ratio 0.2 is 0.7675 - 0.7350 = **0.1075**

For 0.4,the accuracy of all the classes is high , with only a slight distribution in accuracy. This means for this size of data, the classifier is good at classifying all classes to a high accuracy. This is supported with: The *DOA* for ratio 0.4 is 0.8250 - 0.7950 = **0.03**

For ratio 0.6, the accuracy for classes such as kitchen and DVD are very high, meaning this is a good amount of data to give classify items of the class `DVD` and `kitchen`. There is quite a distribution between classes `kitchen` and `book`, meaning that the amount of data is not suitable for classifying items of the class `book`. The *DOA* for ratio 0.6 is 0.8475 - 0.7125 = **0.135**

For 0.8, this is the largest amount of training data tested for this classifier. From the graph, this amount of data is a good amount for an accurate classification of  items for all classes, bar `book`. The *DOA* of this amount of data is 0.8100 - 0.7050 = **0.105**.

In conclusion, from all the calculated *Distribution of accuracies*, we can see that the ratio of **0.4** is the best amount of training data in order to ensure accurate classifications for **all classes**, along with high accuracies of classification. This supports the claim the amount of data does impact the accuracy of classification, with some amounts of data achieveing much higher accuracies overall than others which give higher accuracies to a single domain.

b) For each possible combination of source and target domain, **evaluate** the accuracy of a Naive Bayes classifier trained on the source domain and tested on the target domain.  There are four domains so there are 16 possible combinations you should consider.  You should use a table and an appropriate graph(s) to display your results.  Make sure you **discuss** your results and conclusions. \[8 marks\]

### Experimental Design

The 16 Combinations are:
1. DVD training data with DVD, Book, Kitchen & Electronic testing data
2. Book training data with DVD, Book, Kitchen & Electronic testing data
3. Kitchen training data with DVD, Book, Kitchen & Electronic testing data
4. Electrons training data with DVD, Book, Kitchen & Electronic testing data

The code will calculate the accuracy of each combination and store them into four arrays, one for every domain/class.
These will then be translated into a table and corrosponding graph to help see how the accuracy is affected by being trained on the source, but tested on the target domain.

In [ ]:
import nltk.classify.naivebayes
import numpy as np
NB = nltk.NaiveBayesClassifier
dvdar = []
bookar = []
kitar = []
elecar = []

dvd_train_data, dvd_testing_data = get_training_test_data("dvd")
book_train_data, book_testing_data = get_training_test_data("book")
kitchen_train_data, kitchen_testing_data = get_training_test_data("kitchen")
electronics_train_data, electronics_testing_data = get_training_test_data("electronics")

dvdar.append(nltk.classify.accuracy(NB.train(dvd_train_data), dvd_testing_data))
dvdar.append(nltk.classify.accuracy(NB.train(dvd_train_data), book_testing_data))
dvdar.append(nltk.classify.accuracy(NB.train(dvd_train_data), kitchen_testing_data))
dvdar.append(nltk.classify.accuracy(NB.train(dvd_train_data), electronics_testing_data))

bookar.append(nltk.classify.accuracy(NB.train(book_train_data), dvd_testing_data))
bookar.append(nltk.classify.accuracy(NB.train(book_train_data), book_testing_data))
bookar.append(nltk.classify.accuracy(NB.train(book_train_data), kitchen_testing_data))
bookar.append(nltk.classify.accuracy(NB.train(book_train_data), electronics_testing_data))

kitar.append(nltk.classify.accuracy(NB.train(kitchen_train_data), dvd_testing_data))
kitar.append(nltk.classify.accuracy(NB.train(kitchen_train_data), book_testing_data))
kitar.append(nltk.classify.accuracy(NB.train(kitchen_train_data), kitchen_testing_data))
kitar.append(nltk.classify.accuracy(NB.train(kitchen_train_data), electronics_testing_data))

elecar.append(nltk.classify.accuracy(NB.train(electronics_train_data), dvd_testing_data))
elecar.append(nltk.classify.accuracy(NB.train(electronics_train_data), book_testing_data))
elecar.append(nltk.classify.accuracy(NB.train(electronics_train_data), kitchen_testing_data))
elecar.append(nltk.classify.accuracy(NB.train(electronics_train_data), electronics_testing_data))

data = [['DVD', dvdar[0], dvdar[1], dvdar[2], dvdar[3]], ['Book', bookar[0], bookar[1], bookar[2], bookar[3]], ['Kitchen',kitar[0], kitar[1], kitar[2], kitar[3]],  ['Electronics', elecar[0], elecar[1], elecar[2], elecar[3]]]
df = pd.DataFrame(data,columns=['Domain','DVD', 'Book', 'Kitchen', 'Electronics'])
display(df)

barWidth = 0.2
fig, ax = plt.subplots()

b = np.arange(len(dvdar))
x1 = ax.bar(b, dvdar, barWidth, bottom=0)
x2 = ax.bar(b + barWidth, bookar, barWidth, bottom=0)
x3 = ax.bar(b + barWidth*2, kitar, barWidth, bottom=0)
x4 = ax.bar(b + barWidth*3, elecar, barWidth, bottom=0)

ax.set_title('Accuracy of a classifier trained on the source domain and tested on the target domain')
ax.set_xticks(b + barWidth)
ax.set_xticklabels(('DVD', 'Book', 'Kitchen', 'Electronics'))

ax.legend((x1[0], x2[0], x3[0], x4[0]), ('DVD', 'Book', 'Kitchen', 'Electronics'))
ax.set_ylabel('Accuracy')
ax.set_xlabel('Source Domain')
ax.autoscale_view()
plt.rcParams["figure.figsize"] = [20,9]
plt.show()

### Discussion of Results

As suspected, by training it the source domain, the target domain with the highest accuracy tends to same as the source domain. This is evidently present in source domains: `DVD & Kitchen`. As the data is trained source domain, the highest accuracy should of target domain should be the same as the source domain. Moreover even in source domains such as `Book & Electronics` the corrosponding target domains have the second highest accuracy. 

What the results further show is that the target domain of `DVD` has a high accuracy of classifying for all of the source domains, meaning that accurate classification of DVD target domains is achieveable on all source domains present. This is not the case for target domains such as `Book` however as much less accurate classification takes place on source domains that are not its own.

In conclusion, for some classes such as DVD, the class of the source domain does not matter, as high accuracies are achieveable on all domains, while more frequently, a classifier of *x* source domain will have the highest accuracy of classification for *x* target domain.

c) Adapt your code so that you can build and use a training set built from multiple categories.  Now **investigate** how having a mixture of source domains affects the accuracy of the Naive Bayes classifier on the target domain.  Make sure you control for or consider how much any improvements are due to the quantity of the training data. \[9 marks\]

### Experimental Design

Creating 4 sets of data, each set with an extra domain than the previous. Each domain will be trained and tested against the different sets. This means each domain will have 4 different values of accuracy, each for a different number of sets.The accuracy values will be stored in arrays, one for each target domain.

In [ ]:
import nltk.classify.naivebayes
import numpy as np
NB = nltk.NaiveBayesClassifier

s1 = [] # Array of all accuracy values at DVD target domain
s2 = [] # Array of all accuracy values at Book target domain
s3 = [] # Array of all accuracy values at Kitchen target domain
s4 = [] # Array of all accuracy values at Electronics target domain

setOf1 = kitchen_train_data
setOf2 = dvd_train_data + kitchen_train_data
setOf3 = dvd_train_data + kitchen_train_data + electronics_train_data
setOf4 = dvd_train_data + kitchen_train_data + electronics_train_data + book_train_data

s1.append(nltk.classify.accuracy(NB.train(setOf1), dvd_testing_data))
s1.append(nltk.classify.accuracy(NB.train(setOf1), book_testing_data))
s1.append(nltk.classify.accuracy(NB.train(setOf1), kitchen_testing_data))
s1.append(nltk.classify.accuracy(NB.train(setOf1), electronics_testing_data))

s2.append(nltk.classify.accuracy(NB.train(setOf2), dvd_testing_data))
s2.append(nltk.classify.accuracy(NB.train(setOf2), book_testing_data))
s2.append(nltk.classify.accuracy(NB.train(setOf2), kitchen_testing_data))
s2.append(nltk.classify.accuracy(NB.train(setOf2), electronics_testing_data))

s3.append(nltk.classify.accuracy(NB.train(setOf3), dvd_testing_data))
s3.append(nltk.classify.accuracy(NB.train(setOf3), book_testing_data))
s3.append(nltk.classify.accuracy(NB.train(setOf3), kitchen_testing_data))
s3.append(nltk.classify.accuracy(NB.train(setOf3), electronics_testing_data))

s4.append(nltk.classify.accuracy(NB.train(setOf4), dvd_testing_data))
s4.append(nltk.classify.accuracy(NB.train(setOf4), book_testing_data))
s4.append(nltk.classify.accuracy(NB.train(setOf4), kitchen_testing_data))
s4.append(nltk.classify.accuracy(NB.train(setOf4), electronics_testing_data))


data = [['DVD', s1[0], s1[1], s1[2], s1[3]], ['Book', s2[0], s2[1], s2[2], s2[3]], ['Kitchen',s2[0], s2[1], s2[2], s2[3]],  ['Electronics', s3[0], s3[1], s3[2], s3[3]]]
df = pd.DataFrame(data,columns=['Domain','1 Set', '2 Sets', '3 Sets', '4 Sets'])
display(df)

barWidth = 0.2
fig, ax = plt.subplots()

b = np.arange(len(s1))
x1 = ax.bar(b, s1, barWidth, bottom=0)
x2 = ax.bar(b + barWidth, s2, barWidth, bottom=0)
x3 = ax.bar(b + barWidth*2, s3, barWidth, bottom=0)
x4 = ax.bar(b + barWidth*3, s4, barWidth, bottom=0)

ax.set_title('How a mixture of source domains affects the accuracy of a classifier')
ax.set_xticks(b + barWidth)
ax.set_xticklabels(('1', '2', '3', '4'))

ax.legend((x1[0], x2[0], x3[0], x4[0]), ('DVD', 'Book', 'Kitchen', 'Electronics'))
ax.set_ylabel('Accuracy')
ax.set_xlabel('Number of Source Domains')
ax.autoscale_view()
plt.rcParams["figure.figsize"] = [20,9]
plt.show()

### Discussion of results

By increasing the number of source domains, the accuracy of the classification on target domains increases. This is conveyed for target domain of any of the domains. All accuracies increase from having one source domains to four, meaning that using kitchen training data as a base source domain (a part of all sets of domains), this is the case. Here i have only shown 1 possibility, as there are 4^4 combinations (256) but i have only shown 1. Another interesting point to make is that for 3 source domains, the accuracy for each target domain is much higher. This would suggest that the combination of the three sets allows for accurate classification of all target domains.

Use the code below to verify that the length of your submission does not exceed 2000 words.

In [ ]:
##This code will word count all of the markdown cells in the notebook saved at filepath
##Running it before providing any answers shows that the questions have a word count of 1282

import io
from nbformat import current
filepath="a1.ipynb"
question_count=1282
with io.open(filepath, 'r', encoding='utf-8') as f:
    nb = current.read(f, 'json')
word_count = 0
for cell in nb.worksheets[0].cells:
    if cell.cell_type == "markdown":
        word_count += len(cell['source'].replace('#', '').lstrip().split(' '))
print("Submission length is {}".format(word_count-question_count))